In [ ]:
import json
import os
import sys
from litellm import completion
from typing import List, Dict
import traceback
import time
from dataclasses import dataclass



'''https://console.groq.com/keys'''

In [ ]:

os.environ['GROQ_API_KEY'] = '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'


In [ ]:


#GOAL
@dataclass(frozen=True)
class Goal:
    priority: int
    name: str
    description: str




#ACTION and ACTIONREGISTRY
class Action:
    def __init__(self, name, function, description, parameters, terminal=False):
        self.name = name
        self.function = function
        self.description = description
        self.parameters = parameters
        self.terminal = terminal

    def execute(self, **kwargs):
        return self.function(**kwargs)

class ActionRegistry:
    def __init__(self):
        self.actions = {}

    def register(self, action):
        self.actions[action.name] = action

    def get_action(self, name):
        return self.actions.get(name)





#MEMORY

class Memory:
    def __init__(self):
        self.messages = []

    def add(self, role, content):
        self.messages.append({"role": role, "content": content})

    def get(self):
        return self.messages




#ENVIROMENT
class Environment:
    def execute_action(self, action, args):
        try:
            result = action.execute(**args)
            return {
                "tool_executed": True,
                "result": result,
                "timestamp": time.strftime("%Y-%m-%dT%H:%M:%S%z")
            }
        except Exception as e:
            return {
                "tool_executed": False,
                "error": str(e),
                "traceback": traceback.format_exc()
            }



#AGENT
class Agent:
    def __init__(self, goal, registry, environment, generate_response):
        self.goal = goal
        self.registry = registry
        self.environment = environment
        self.generate_response = generate_response

    def run(self, user_input, memory, max_iterations=10):
        memory.add("user", user_input)

        for _ in range(max_iterations):
            prompt = agent_rules + memory.get()
            print("Agent thinking...")
            response = self.generate_response(prompt)
            print(f"Agent response: {response}")

            action_data = parse_action(response)
            action = self.registry.get_action(action_data["tool_name"])

            if not action:
                result = {"error": "Unknown action"}
            else:
                result = self.environment.execute_action(action, action_data["args"])
                if action.terminal:
                    print(action_data["args"]["message"])
                    break

            memory.add("assistant", response)
            memory.add("user", json.dumps(result))

            print(f"Action result: {result}")


